In [1]:
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df2 = pd.read_csv('data/grouped_audio_billboard_genre_data.csv')

# Drop the null columns where all values are null
df2 = df2.dropna(axis='columns', how='all')

# Drop the null rows
df2 = df2.dropna()
df2.head()

# Display dataframe 
df2.head()

,track_id,genre,spotify_track_explicit,spotify_track_duration_ms,spotify_track_popularity,danceability,energy,key,loudness,mode,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,week_pos,instance,peak_pos,weeks_on_chart
0,"AdictoTainy, Anuel AA & Ozuna",pop,0.0,270740.0,91.0,0.734,0.836,10.0,-4.803,0.0,...,0.01700,0.000016,0.1790,0.623,80.002,4.0,98.0,1.0,86.0,2.0
1,The Ones That Didn't Make It Back HomeJustin M...,country,0.0,0.0,0.0,0.000,0.000,0.0,0.000,0.0,...,0.00000,0.000000,0.0000,0.000,0.000,0.0,100.0,1.0,100.0,1.0
2,ShallowLady Gaga & Bradley Cooper,pop,0.0,215733.0,89.0,0.572,0.385,7.0,-6.362,1.0,...,0.37100,0.000000,0.2310,0.323,95.799,4.0,28.0,1.0,28.0,1.0
3,EnemiesPost Malone Featuring DaBaby,rap,1.0,196760.0,86.0,0.542,0.674,6.0,-4.169,1.0,...,0.05880,0.000000,0.0955,0.667,76.388,4.0,98.0,1.0,16.0,15.0
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo",rap,1.0,228185.0,61.0,0.948,0.623,8.0,-5.725,0.0,...,0.00124,0.000001,0.0716,0.856,135.979,4.0,94.0,1.0,94.0,3.0


In [3]:
track_df = pd.read_csv('data/audio_features_hot_100_1958_2019.csv')
track = track_df[['track_id','artist','track']]
track.drop_duplicates()

,track_id,artist,track
0,"AdictoTainy, Anuel AA & Ozuna","Tainy, Anuel AA & Ozuna",Adicto
1,The Ones That Didn't Make It Back HomeJustin M...,Justin Moore,The Ones That Didn't Make It Back Home
2,ShallowLady Gaga & Bradley Cooper,Lady Gaga & Bradley Cooper,Shallow
3,EnemiesPost Malone Featuring DaBaby,Post Malone Featuring DaBaby,Enemies
4,"Bacc At It AgainYella Beezy, Gucci Mane & Quavo","Yella Beezy, Gucci Mane & Quavo",Bacc At It Again
...,...,...,...
28487,"Tonight, TonightThe Smashing Pumpkins",The Smashing Pumpkins,"Tonight, Tonight"
28488,Tonight's The Night (Gonna Be Alright)Rod Stewart,Rod Stewart,Tonight's The Night (Gonna Be Alright)
28489,Tonight's The NightBLACKstreet,BLACKstreet,Tonight's The Night
28490,Tonight's The NightS.S.O.,S.S.O.,Tonight's The Night


In [4]:
df2['genre'].unique()

array(['pop', 'country', 'rap', 'hiphop', 'other', 'latin', 'house',
       'folk', 'r&b', 'adult standards', 'rock', 'metal', 'show tunes',
       'soul', 'jazz'], dtype=object)

In [5]:
df2['genre'] = df2['genre'].replace('pop',float(0)).replace('country',float(1))\
.replace('hiphop',float(2)).replace('other',float(3)).replace('latin',float(3)).replace('latin',float(4))\
.replace('house',float(5)).replace('folk',float(6)).replace('r&b',float(7)).replace('adult standards',float(8))\
.replace('rock',float(9)).replace('metal',float(10)).replace('show tunes',float(11)).replace('soul',float(12))\
.replace('rap',float(13)).replace('jazz',float(14))

In [6]:
from sklearn.model_selection import train_test_split
# y = pd.Series(df2["target"])
X = df2.drop(columns=["track_id" ,'peak_pos']) #spotify_track_durations
X_scaled = df2.drop(columns=["track_id" ,'peak_pos']) #spotify_track_durations





In [7]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X)


X_scaled = X_scaler.transform(X)


In [8]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=120)
kmeans2 = KMeans(n_clusters=120)

# Fit the model to the data
kmeans.fit(X)
kmeans2.fit(X_scaled)
# Use the data to predict the clusters
# save the predictions as `predicted_clusters`
predicted_clusters = kmeans.predict(X)
predicted_clusters_scaled = kmeans2.predict(X_scaled)

In [9]:
df2['cluster'] = predicted_clusters
df2['cluster_scaled'] = predicted_clusters_scaled

In [10]:
cluster_agg = df2.groupby(by = 'cluster')
pd.DataFrame(cluster_agg['track_id'].count()).sort_values('track_id',ascending = False)

,track_id
cluster,
1,4740
28,467
87,456
93,449
55,433
...,...
102,1
12,1
16,1


In [11]:
centers = kmeans.cluster_centers_
pd.DataFrame(centers).to_csv('data/kmeans_centers.csv')

clusters = df2[['track_id','cluster','peak_pos']].merge(track,on='track_id',how='inner').drop_duplicates()
clusters

clusters.to_csv('data/kmeans_clusters.csv')
# centersdf = pd.DataFrame(centers)#.to_csv('kmeans_centers.csv')

In [12]:
centers_scaled = kmeans2.cluster_centers_
pd.DataFrame(centers_scaled).to_csv('data/kmeans_centers_scaled.csv')

clusters_scaled = df2[['track_id','cluster_scaled','peak_pos']].merge(track,on='track_id',how='inner').drop_duplicates()
clusters

clusters.to_csv('data/kmeans_clusters_scaled.csv')
# centersdf = pd.DataFrame(centers)#.to_csv('kmeans_centers.csv')